# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [27]:
import sys
import os
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

import pandas as pd
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [28]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])
inatstatuses = filter_state_statuses("Tasmania|TAS", ".tas.gov")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
212,170159,1007040,702203,6829,,Threatened Species Protection Act 1995,EN,https://www.threatenedspecieslink.tas.gov.au/P...,,,...,Dianella,amoena,,2021-07-27T17:12:17Z,Dianella amoena,species,http://www.catalogueoflife.org/annual-checklis...,,,
2558,160921,1038747,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Ranunculus...,,obscured,...,Ranunculus,diminutus,,2022-06-13T17:38:59Z,Ranunculus diminutus,species,,,,
2547,160910,1038906,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Myosurus-a...,,obscured,...,Myosurus,australis,,2020-08-19T03:33:32Z,Myosurus australis,species,,,,
933,160738,1038964,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Boronia%20...,,obscured,...,Boronia,hemichiton,,2020-08-17T05:51:50Z,Boronia hemichiton,species,,,,
2557,160920,1038976,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pimelea-ax...,,obscured,...,Pimelea,axiflora,axiflora,2020-12-02T04:00:26Z,Pimelea axiflora axiflora,subspecies,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,160759,958863,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Corunastyl...,,,...,Genoplesium,nudiscapum,,2022-10-26T09:34:12Z,Genoplesium nudiscapum,species,https://eol.org/pages/1099120,,,
2594,161103,960004,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Thelymitra...,,obscured,...,Thelymitra,bracteata,,2022-07-14T00:09:44Z,Thelymitra bracteata,species,http://www.catalogueoflife.org/annual-checklis...,,,
3380,268029,993886,1116633,6829,,TASMANIAN THREATENED SPECIES LISTING STATEMENT,EN,https://nre.tas.gov.au/Documents/Prasophyllum%...,1. total population estimated to number fewer ...,obscured,...,Prasophyllum,abblittiorum,,2020-02-18T23:45:10Z,Prasophyllum abblittiorum,species,https://eol.org/pages/51504614,,,
813,160748,993902,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Caladenia-...,,,...,Caladenia,dienema,,2020-12-15T00:50:01Z,Caladenia dienema,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [29]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists
Sensitive list: `geoprivacy` = `obscured`
overrides

In [30]:

# %%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally
# Download lists data. Retrieve binomial and trinomial names from GBIF. Save locally to CSV

sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr491?max=10000&includeKVP=true")
# sensitivelist = lf.kvp_to_columns(sensitivelist)   # kvp values null
sensitivelist.to_csv(sourcedir + "tas-ala-sensitive.csv", index=False)

conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr654?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "tas-ala-conservation.csv", index=False)

In [31]:
# Read sensitive list data
sensitivelist = pd.read_csv(sourcedir + "tas-ala-sensitive.csv", dtype=str)
sensitivelist['status'] = 'sensitive'
sensitivelist['bionet_geoprivacy'] = 'obscured'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,status,bionet_geoprivacy
0,24386,Lomatia tasmanica,King's Lomatia,Lomatia tasmanica,https://id.biodiversity.org.au/node/apni/2903191,dr491,[],sensitive,obscured


In [32]:
# Read conservation list data
conservationlist = pd.read_csv(sourcedir + "tas-ala-conservation.csv", dtype=str)
conservationlist['bionet_geoprivacy'] = 'open'
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,scientificNameAuthorship,vernacular name,family,class,sourceStatus,status,sourceKingdom,kingdom,speciesGroup,sourceOrigin,higherGeography,E P B C A_status,bionet_geoprivacy
0,4033150,Costora iena,Caddis Fly (great Lakes),Costora iena,https://biodiversity.org.au/afd/taxa/7de808fb-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Mosely,Caddis Fly (Great Lakes),Trichoptera,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
1,4033128,Hadronyche pulvinator,Cascade Funnel-web Spider,Hadronyche pulvinator,https://biodiversity.org.au/afd/taxa/2acd0df1-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hickman,Cascade Funnel-web Spider,Araneae,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
2,4033378,Hypolimnus pedderensis,Lake Pedder Earthworm,Hypolimnus pedderensis,https://biodiversity.org.au/afd/taxa/7db09aa3-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Jamieson),Lake Pedder Earthworm,Haplotaxida,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,EX,open
3,4033234,Thylacinus cynocephalus,Thylacine,Thylacinus cynocephalus,https://biodiversity.org.au/afd/taxa/97764bed-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Harris,Thylacine,Mammals,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,end,endemic Tasmania,EX,open
4,4033416,Gallirallus philippensis macquariensis,NaN,Gallirallus philippensis macquariensis,NZOR-6-101588,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hutton,Macquarie Island Rail,Birds,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,eMI,endemic Macquarie Island,EX,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,4033369,Erioderma sorediatum,NaN,Erioderma sorediatum,ALA_DR654_679,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",D.J.Galloway & P.M.Jørg.,NaN,Pannariaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
679,4033419,Xanthoparmelia amphixantha,NaN,Xanthoparmelia amphixantha,ALA_DR654_680,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Müll.Arg.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
680,4033291,Xanthoparmelia molliuscula,NaN,Xanthoparmelia molliuscula,ALA_DR654_681,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Ach.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
681,4033236,Xanthoparmelia subloxodella,NaN,Xanthoparmelia subloxodella,ALA_DR654_682,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...","(Elix & Kantvilas) O.Blanco, A.Crespo, Elix, D...",NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open


In [33]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['name','bionet_geoprivacy', 'status', 'lsid']],
                    conservationlist[['name','bionet_geoprivacy', 'status', 'lsid']]]).drop_duplicates()
statelist

,name,bionet_geoprivacy,status,lsid
0,Lomatia tasmanica,obscured,sensitive,https://id.biodiversity.org.au/node/apni/2903191
0,Costora iena,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/7de808fb-...
1,Hadronyche pulvinator,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/2acd0df1-...
2,Hypolimnus pedderensis,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/7db09aa3-...
3,Thylacinus cynocephalus,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/97764bed-...
...,...,...,...,...
678,Erioderma sorediatum,open,Endangered,ALA_DR654_679
679,Xanthoparmelia amphixantha,open,Endangered,ALA_DR654_680
680,Xanthoparmelia molliuscula,open,Endangered,ALA_DR654_681
681,Xanthoparmelia subloxodella,open,Endangered,ALA_DR654_682


In [34]:
# check for duplicates with conflicting information
statelist.groupby('name').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)


,name,bionet_geoprivacy,status,lsid
0,Lomatia tasmanica,obscured,sensitive,https://id.biodiversity.org.au/node/apni/2903191
569,Lomatia tasmanica,open,Endangered,https://id.biodiversity.org.au/node/apni/2903191


### 4. Equivalent IUCN statuses

In [35]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild', 'Extinct'}
statelist.groupby(['status'])['status'].count()

status
Endangered          222
Presumed Extinct     28
Rare                311
Vulnerable          122
sensitive             1
Name: status, dtype: int64

In [36]:
iucnStatusMappings = {
    'Presumed Extinct':'Extinct',
    'Rare':'Vulnerable',
    'sensitive':'Vulnerable'
}

### 5. Determine best place ID to use

In [37]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6825

place_id  place_name          place_display_name    
6744      Australia           Australia                  2
6829      Tasmania            Tasmania, AU              77
9994      Northern Territory  Northern Territory, AU     6
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [38]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['name', 'bionet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='name',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['name'])
mergedstatuses


,name,bionet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,status,geoprivacy,place_id,place_display_name
216,Acacia axillaris,open,https://id.biodiversity.org.au/node/apni/2911902,160726,Acacia axillaris,775192,717122,,30,Environment Protection and Biodiversity Conser...,VU,obscured,6744,Australia
300,Acacia pataczekii,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,Acacia siculiformis,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Acacia ulicifolia,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Acacia uncifolia,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Xerochrysum palustre,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,Zearaja maugeana,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,Zieria littoralis,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678,Zieria veronicea subsp. veronicea,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Threatened Species Protection Act 1995"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.threatenedspecieslink.tas.gov.au"
mergedstatuses['new_url'] = mergedstatuses['lsid']
mergedstatuses['new_geoprivacy'] = "obscured"
mergedstatuses['new_place_id'] = '6829'  # Tasmania
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Threatened')
mergedstatuses

,name,bionet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,...,place_id,place_display_name,new_authority,new_description,new_url,new_geoprivacy,new_place_id,new_username,new_iucn_equivalent,new_status
216,Acacia axillaris,open,https://id.biodiversity.org.au/node/apni/2911902,160726,Acacia axillaris,775192,717122,,30,Environment Protection and Biodiversity Conser...,...,6744,Australia,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2911902,obscured,6829,peggydnew,Vulnerable,VU
300,Acacia pataczekii,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2913609,obscured,6829,peggydnew,Vulnerable,Threatened
355,Acacia siculiformis,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2888960,obscured,6829,peggydnew,Vulnerable,Threatened
356,Acacia ulicifolia,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2907350,obscured,6829,peggydnew,Vulnerable,Threatened
357,Acacia uncifolia,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2916270,obscured,6829,peggydnew,Vulnerable,Threatened
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Xerochrysum palustre,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2890165,obscured,6829,peggydnew,Vulnerable,Threatened
173,Zearaja maugeana,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://biodiversity.org.au/afd/taxa/0d9d194f-...,obscured,6829,peggydnew,Vulnerable,Threatened
522,Zieria littoralis,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2888662,obscured,6829,peggydnew,Vulnerable,Threatened
678,Zieria veronicea subsp. veronicea,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2911634,obscured,6829,peggydnew,Vulnerable,Threatened


## Updates

In [40]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
216,UPDATE,Acacia axillaris,160726,775192,VU,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2911902,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
604,UPDATE,Alternanthera denticulata,160727,369665,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5126...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
605,UPDATE,Amphibromus macrorhinus,160728,1121410,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2916650,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
243,UPDATE,Anogramma leptophylla,162081,180646,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5129...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
360,UPDATE,Aphelia gracilis,163877,1143399,NT,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2899707,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
...,...,...,...,...,...,...,...,...,...,...,...,...
666,UPDATE,Thelymitra benthamiana,161102,520289,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5141...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
667,UPDATE,Thelymitra bracteata,161103,960004,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5141...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
671,UPDATE,Thelymitra malvina,161104,406524,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5141...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
674,UPDATE,Veronica notabilis,161105,791936,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2890187,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...


## No status in iNaturalist via straight scientificName match
The ACT records that didn't match up to a status in iNaturalist

In [41]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,name,bionet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acacia siculiformis,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acacia ulicifolia,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Acacia uncifolia,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Acanthiza pusilla archibaldi,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,Xerochrysum palustre,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,Zearaja maugeana,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,Zieria littoralis,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,Zieria veronicea subsp. veronicea,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,name,bionet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references


In [43]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description


In [44]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "tas.csv", index=False)

In [45]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,name,bionet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acacia siculiformis,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acacia ulicifolia,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Acacia uncifolia,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Acanthiza pusilla archibaldi,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,Xerochrysum palustre,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,Zearaja maugeana,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,Zieria littoralis,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,Zieria veronicea subsp. veronicea,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### are there any that need to be removed?
TAS sensitive list count: 1
TAS conservation list count: 683
TAS inat statuses count: 0

updates to inat status: 67
additional inat status: 0
TAS statuses we can't find a taxon match for in iNaturalist: 617
total: (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over:  that may need checking

In [46]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
d = {'Sensitive': [numsensitive],
    'Conservation': [numconservation],
    'Updates': [numupdates],
    'Additions': [numadditions],
    'No Inat Status': [numnoinatstatus],
    'Unknown to Inat': [numunknownToInat]}
statsdf = pd.DataFrame(data=d)
statsdf

,Sensitive,Conservation,Updates,Additions,No Inat Status,Unknown to Inat
0,1,683,67,0,617,617


In [47]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]


,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
2557,160920,1038976,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pimelea-ax...,,obscured,...,Pimelea,axiflora,axiflora,2020-12-02T04:00:26Z,Pimelea axiflora axiflora,subspecies,,,,
2566,160929,1092444,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Spyridium-...,,obscured,...,Spyridium,eriocephalum,eriocephalum,2021-11-09T06:11:14Z,Spyridium eriocephalum eriocephalum,variety,,,,
3353,267438,1413424,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pneumatopt...,,obscured,...,Pakau,pennigera,,2022-09-25T23:02:37Z,Pakau pennigera,species,https://shop.brit.org/A-Generic-Classification...,,,
1251,152435,19250,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Polytelis,alexandrae,,2019-08-27T01:09:01Z,Polytelis alexandrae,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
2599,161108,323791,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Zieria-ver...,,obscured,...,Zieria,veronicea,veronicea,2021-12-03T03:55:21Z,Zieria veronicea veronicea,subspecies,http://www.ubio.org/browser/details.php?nameba...,,,
904,160736,353835,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum-n...,,obscured,...,Doodia,aspera,,2021-02-22T18:52:06Z,Doodia aspera,species,http://www.ubio.org/browser/details.php?nameba...,,,
1249,152433,38633,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Bellatorias,obiri,,2019-04-30T15:19:21Z,Bellatorias obiri,species,http://reptile-database.reptarium.cz/search.ph...,,,
932,160737,401378,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum%2...,,obscured,...,Doodia,caudata,,2021-02-15T17:49:20Z,Doodia caudata,species,,,,
959,160741,405825,717122,6829,,Threatened Species Protection Act 1995,Presumed extinct,https://dpipwe.tas.gov.au/Documents/Botrychium...,,obscured,...,,,,,Sceptridium australe,,,parsley fern,False,[400050]
1252,152436,40743,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,,,,,Hipposideros stenotis,,,Narrow-eared Roundleaf Bat,False,[1431118]
